# Option strategies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, ToggleButtons, FloatSlider
from IPython.display import display

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Enter extreme values for stock price 
min_price = 0
max_price = 130
n_pts     = 10001

# Maximum price of option
max_option_price = 30

# Maximum strike price
max_strike = 100

# build an array of possible stock prices 
data = pd.DataFrame(np.linspace(min_price, 
                                max_price, 
                                n_pts), 
                     columns=['profit'])

# Consistency check
if np.max(data.profit) < max_option_price:
    raise ValueError(f'Maximum option price ${max_option_price} > maximum stock price ${np.max(data.profit)}')

In [3]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_option(position, option, strike, S_T, opt_price): 
    '''
    Plots profit curve for an option -
    position: long or short
    option: call or put
    S_T : Stock price at expiration
    opt_price: otion price
    ''' 
    
    if option == 'call':
        data['longcall'] = np.where(data['profit'] >= strike, 
                                    data['profit'] - strike - opt_price, 
                                    -opt_price)
        pft = np.where(S_T >= strike, S_T - strike - opt_price, -opt_price)
        if position == 'short':
            data['shortcall'] = -data.longcall
            pft = -pft
    else : # if put
        data['longput'] = np.where(data['profit'] <= strike, 
                                   -data['profit'] + strike - opt_price, 
                                   -opt_price) 
        pft = np.where(S_T <= strike, -S_T + strike - opt_price, -opt_price) 
        if position == 'short':
            data['shortput'] = -data.longput
            pft = -pft
    
    # Plot profits
    axis = data.plot(x = 'profit', 
                     y = position + option, 
                     figsize = (16, 8), 
                     linewidth = 3,
                     label = 'profit')
    # Plot horizontal 0 line
    plt.axhline(0, 
                color = colors[3], 
                linestyle = 'dashed', 
                linewidth = 1, 
                label = 'break-even')
    # Plot terminal price
    plt.axvline(S_T, 
                color = colors[5], 
                linestyle = 'dashdot', 
                label = 'terminal price')
    # Plot strike pice
    plt.axvline(strike, 
                color = colors[2], 
                linestyle = 'dotted', 
                linewidth = 2, 
                label = 'strike price')

    plt.title(position.capitalize() + ' ' + option.lower(), fontsize=30)

    plt.xlabel('Terminal stock price ($)', fontsize=15)
    # Add profit amount to plot
    plt.annotate(f'${pft}', 
                 (min_price-5, pft), 
                 color=colors[0], 
                 fontsize=14,
                 arrowprops=dict(facecolor=colors[0], shrink=0.05),
                 rotation=0)
    
    plt.ylim([-max_strike, max_strike])
    plt.ylabel('Profit ($)', fontsize=15)
    # Style grid
    plt.grid(which='major', color='#d3d3d3', linestyle='-', linewidth=1)
    plt.minorticks_on()
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.1)

    plt.legend()

In [4]:
# prevent description from being cut
string_style = {'description_width': 'initial'}
plot_controls = interactive(plot_option, 
                            position = ToggleButtons(options=['long', 'short'],
                                                     value='long',
                                                     description='Position:',
                                                     disabled=False,
                                                     ),
                            option = ToggleButtons(options=['call', 'put'],
                                                   value='call',
                                                   description='Option:',
                                                   disabled=False
                                                   ),
                            strike = FloatSlider(min=0, 
                                                 max=max_strike, 
                                                 step=.5, 
                                                 value=max_strike/2,
                                                 description='Strike price:',
                                                 disabled=False,
                                                 continuous_update=False,
                                                 readout=True,
                                                 readout_format='.1f',
                                                 ),
                            opt_price = FloatSlider(min=0, 
                                                    max=max_option_price, 
                                                    step=.25, 
                                                    value=max_option_price/2,
                                                    description='Option price:',
                                                    disabled=False,
                                                    continuous_update=False,
                                                    readout=True,
                                                    readout_format='.2f',
                                                    ),
                            S_T = FloatSlider(min=0,
                                              max=max_strike,
                                              step=.5,
                                              value=max_strike/2 + 5,
                                              description='Terminal price:',
                                              style=string_style,
                                              disabled=False,
                                              continuous_update=False,
                                              readout=True,
                                              readout_format='.1f',
                                              )
                            )

In [5]:
display(plot_controls)

interactive(children=(ToggleButtons(description='Position:', options=('long', 'short'), value='long'), ToggleB…

## Spreads

### Bull spread 
#### long & short call, different strikes, same expiration date

In [16]:
def plot_spread(spread, strike_0, strike_1, opt_price_0, opt_price_1, S_T): 
    '''
    Plots profit curve for a spread -
    spread: bull or bear
    strike_0: strike price for long component of spread
    strike_1: strike price for short component of spread
    opt_price_0: option price for long component of spread
    opt_price_1: option price for short component of spread
    S_T : Stock price at expiration
    
    REM: the value of an option sold is always less than the value of an option bought
    ''' 
    # Long component
    strike    = [strike_0, strike_1] # cannot implement lists in the widget 
    opt_price = [opt_price_0, opt_price_1]
    
    if spread == 'bull':
        # Price of call sold < price call bought
        if opt_price[1] > opt_price[0]:
            opt_price[1] = opt_price[0]

        # Long component of spread
        data['longcall'] = np.where(data['profit'] >= strike[0], 
                                    data['profit'] - strike[0] - opt_price[0], 
                                    -opt_price[0]
                                    )
        # Short component of spread
        data['shortcall'] = np.where(data['profit'] >= strike[1],
                                     -data['profit'] + strike[1] + opt_price[1],
                                     opt_price[1]
                                     )
        data['bull'] = data['longcall'] + data['shortcall']

        pft  = np.where(S_T >= strike[0],  S_T - strike[0] - opt_price[0], -opt_price[0])
        pft += np.where(S_T >= strike[1], -S_T + strike[1] + opt_price[1],  opt_price[1])
        
    elif spread == 'bear':
        # The price of put sold always < than price of put purchased
        if opt_price[1] < opt_price[0]:
            opt_price[1] = opt_price[0]
            
        # Long component of spread
        data['longput'] = np.where(data['profit'] <= strike[1], 
                                   -data['profit'] + strike[1] - opt_price[1], 
                                   -opt_price[1]
                                   ) 
        # Short component of spread
        data['shortput'] = np.where(data['profit'] <= strike[0], 
                                    data['profit'] - strike[0] + opt_price[0], 
                                    opt_price[0]
                                    ) 
        data['bear'] = data['longput'] + data['shortput']
        
        pft  = np.where(S_T <= strike[1], -S_T + strike[1] - opt_price[1], -opt_price[1]) 
        pft += np.where(S_T <= strike[0],  S_T - strike[0] + opt_price[0],  opt_price[0])
        
    
    # Plot profits
    axis = data.plot(x = 'profit', 
                     y = spread, 
                     figsize = (16, 8), 
                     linewidth = 3,
                     label = 'profit')
    # Plot horizontal 0 line
    plt.axhline(0, 
                color = colors[3], 
                linestyle = 'dashed', 
                linewidth = 1, 
                label = 'break-even')
    # Plot terminal price
    plt.axvline(S_T, 
                color = colors[5], 
                linestyle = 'dashdot', 
                label = 'terminal price')
    # Plot strike prices
    plt.axvline(strike[0], 
                color = colors[2], 
                linestyle = 'dotted', 
                linewidth = 2, 
                label = 'strike price 1 (long)')
    plt.axvline(strike[1], 
                color = colors[4], 
                linestyle = 'dotted', 
                linewidth = 2, 
                label = 'strike price 2 (short)')

    plt.title(spread.capitalize() + ' spread', fontsize=30)

    plt.xlabel('Terminal stock price ($)', fontsize=15)
    # Add profit amount to plot
    plt.annotate(f'${pft}', 
                 (min_price-5, pft), 
                 color=colors[0], 
                 fontsize=14,
                 arrowprops=dict(facecolor=colors[0], shrink=0.05),
                 rotation=0)
    
    plt.ylim([-np.max(data.profit), np.max(data.profit)])
    plt.ylabel('Profit ($)', fontsize=15)
    # Style grid
    plt.grid(which='major', color='#d3d3d3', linestyle='-', linewidth=1)
    plt.minorticks_on()
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.1)

    plt.legend()

In [17]:
plot_controls = interactive(plot_spread,
                            spread = ToggleButtons(options=['bull', 'bear'],
                                                   value='bull',
                                                   description='Position:',
                                                   disabled=False
                            ),
                            strike_0 = FloatSlider(min=0, 
                                                   max=max_strike, 
                                                   step=.5, 
                                                   value=max_strike/2,
                                                   description='Strike price (long):',
                                                   style=string_style,
                                                   disabled=False,
                                                   continuous_update=False,
                                                   readout=True,
                                                   readout_format='.1f',
                                                   ),
                            strike_1 = FloatSlider(min=0, 
                                                   max=max_strike, 
                                                   step=.5, 
                                                   value=max_strike/2+5,
                                                   description='Strike price (short):',
                                                   style=string_style,
                                                   disabled=False,
                                                   continuous_update=False,
                                                   readout=True,
                                                   readout_format='.1f',
                                                   ),
                            opt_price_0 = FloatSlider(min=0, 
                                                    max=max_option_price, 
                                                    step=.25, 
                                                    value=max_option_price/2,
                                                    description='Option price (long):',
                                                    style=string_style,
                                                    disabled=False,
                                                    continuous_update=False,
                                                    readout=True,
                                                    readout_format='.2f',
                                                    ),
                            opt_price_1 = FloatSlider(min=0, 
                                                            max=max_option_price, 
                                                            step=.25, 
                                                            value=max_option_price/2-5,
                                                            description='Option price (short):',
                                                            style=string_style,
                                                            disabled=False,
                                                            continuous_update=False,
                                                            readout=True,
                                                            readout_format='.2f',
                                                            ),
                            S_T = FloatSlider(min=0,
                                                      max=max_strike,
                                                      step=.5,
                                                      value=max_strike/2 + 5,
                                                      description='Terminal price:',
                                                      style=string_style,
                                                      disabled=False,
                                                      continuous_update=False,
                                                      readout=True,
                                                      readout_format='.1f',
                                                      )
                                  )

In [18]:
display(plot_controls)

interactive(children=(ToggleButtons(description='Position:', options=('bull', 'bear'), value='bull'), FloatSli…